In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
import re

In [2]:
def get_job_info(trabajo_link):
    """El input es un enlace que dirige a una única oferta de trabajo. Devuelve un tuple con la información más relevante: descripción del anuncio y del trabajo"""
    anuncio_list = []
    description_list = []
    import requests
    from bs4 import BeautifulSoup
    try:
        response = get_page(trabajo_link)
        result_page = BeautifulSoup(response.content,'lxml')
        anuncio_list = get_anuncio_list(result_page)
        description_list = get_job_description(result_page)
        empresa_name = get_name_empresa(result_page)
        output = unify_lists([empresa_name, anuncio_list, description_list])
        return output
    except:
        return None

In [3]:
def get_page(url):
    response = requests.get(url)
    if not response.status_code == 200:
        print("It didn't work")
    return response

In [4]:
def get_anuncio_list(result_page):
    '''This function recover the information concerning ID Empleo, Categoria, Ubicación, etc'''
    anuncio_list = []
    for info in result_page.find_all('div', class_='smallListingInfo'):
        if not 'Primera' in info.get_text(): #Exclude 'Primera Publicacion'
            if not 'Contrato' in info.get_text():
                anuncio_item = info.get_text()
                anuncio_item = anuncio_item[anuncio_item.find(':')+2:]
                anuncio_list.append(anuncio_item.replace('\n',''))  #Get items without \n
                #anuncio_list.append(info.get_text().replace('\n',''))
    return anuncio_list

In [5]:
def get_job_description(result_page):
    text = ''
    for paragraph in result_page.find_all('p'):
        if paragraph.text != "\n":
            text = text + paragraph.text
    text = clean_text(text)
    return text

In [6]:
def get_name_empresa(result_page):
    Enterprise_information = result_page.find('div', class_ = 'compProfileInfo')
    return Enterprise_information.find('strong').get_text()

In [7]:
def clean_text(text):
    import unidecode
    text = text.lower().replace('\n','') #Get items without \n
    text = re.sub("(\(\w+\))", '', text) #Delete parenthesis
    text = unidecode.unidecode(text) #Delete accents
    text = re.sub('[^A-Za-z0-9\s]','',text) #Delete anything but letters, numbers and spaces
    return text

In [8]:
def unify_lists(job_list):
    new_list = []
    new_list.append(job_list[0])
    for element in job_list[1]:
        new_list.append(element)
    new_list.append(job_list[2])
    return new_list

In [9]:
trabajo_link = "https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695472/Analista-de-Tesorería.html?searchId=1545862649.53&page=1"
response = get_page(trabajo_link)
result_page = BeautifulSoup(response.content,'lxml')
anuncio_list = get_anuncio_list(result_page)
#description_list = get_job_description(result_page)
#empresa_name = get_name_empresa(result_page)
#output = unify_lists([empresa_name, anuncio_list, description_list])

In [10]:
def trabajopolis(job_description):
    import time
    """ A partir de una (o más) palabras claves  """
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    for job in job_description:
        url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="+ job + "&page=1&listings_per_page=1000"
        response = requests.get(url)
        if response.status_code == 200:
            print("The request response cycle was successful")
            results_page = BeautifulSoup(response.content,'lxml')
            #trabajo_list = list()
            trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
            for anuncio in trabajos:
                trabajo_link = anuncio.find('a').get('href')
                trabajo_name = anuncio.find('b').get_text()
                print(trabajo_link)
             #   trabajo_list.append((trabajo_name,trabajo_link))
            #for trabajo in trabajo_list:
                output = get_job_info(trabajo_link)
                trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
        else:
            print("The request response cycle work didn't work")
        time.sleep(10) 
    return trabajos_list

In [11]:
def trabajopolis2():
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    url = "https://www.trabajopolis.bo/search-results-jobs/?searchId=1546400657&action=search&page=1&listings_per_page=1000"
    response = requests.get(url)
    if response.status_code == 200:
        print("The request response cycle was successful")
        results_page = BeautifulSoup(response.content,'lxml')
        trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
        for anuncio in trabajos:
            trabajo_link = anuncio.find('a').get('href')
            trabajo_name = anuncio.find('b').get_text()
            print(trabajo_name)
            output = get_job_info(trabajo_link)
            trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
    else:
        print("The request response cycle work didn't work")
    return trabajos_list

In [12]:
Results = trabajopolis2()

The request response cycle was successful
Datawarehouse Engineer
Analista de Riesgo Financiero Sénior – La Paz
Analista Financiero(a) - La Paz
Encargado(a) de Inventarios
Encargado(a) de Sucursal
Asesoras(es) Ejecutivas(os)
Encargado(a) de Unidad de Facturación
Supervisor(a) Comercial - La Paz
Responsable de Recepción, Revisión y Despacho de Mercadería - Santa Cruz
Auxiliar de Almacén - Santa Cruz
Encargado(a) de Plataforma - La Paz
Oficial de Créditos - La Paz
Secretaria - La Paz
Cajero(a) (La Paz - Agencia Mapiri)
Oficial de Créditos Júnior - Chuquisaca
Asesor(a) Comercial - Santa Cruz
Pasante para el Área Administrativa
Oficial de Fianzas y Cauciones - Santa Cruz
Asistente de Emisión - Santa Cruz
Contador(a) General - Santa Cruz
Analista de Importaciones - Santa Cruz
Jefe de Ventas
Supervisor(a) de Despachos y Distribución - Santa Cruz
Ejecutivo(a) de Ventas - Santa Cruz
Vendedor(a) en Provincias - Santa Cruz
Analista de Riesgo - Santa Cruz
Auxiliar Contable
Cajero(a) - Sucre
Analis

Inspector(a) de Siniestros FISO - La Paz
Analista de Mercado
Contador(a) General
Ejecutivo(a) de Ventas - El Alto
Asistente de Monitoreo de Sistemas - Oficina Nacional
Asistente de Operaciones y Administración - Tarija
Cajero(a) - Chuquisaca
Auditor(a) Interno(a) - Normativa - Oficina Nacional
Backend Software Engineer
Support Specialist
Customer Success Agent
Ingeniero(a) de Software
Mensajero(a)
Encargado(a) de Ejecución de Obra y Técnica
Coordinador(a) de Ventas Interior
Encargado(a) de Importaciones y Compras - La Paz
Vendedores(as) Corporativos(as) B2B - La Paz
Contador(a)
Encargado(a) de Control de Calidad y Procesos - Warnes
Asesor(a) Comercial - Santa Cruz
Ejecutivo(a) de Cobranza
Gerente Comercial
Ejecutiva(o) de Ventas
Vendedor(a) de Repuestos - Santa Cruz
Asesor(a) de Ventas en Provincias
Asesor(a) de Ventas Retail
Jefe Comercial - Vehículos Livianos - La Paz
Ejecutivo(a) Comercial - Santa Cruz
Asesor(a) Comercial - La Paz
Auxiliar Comercial - Santa Cruz
Coordinador(a) de RS

Cobrador(a)
Ejecutivo(a) de Marketing - Santa Cruz
Asistente Administrativo(a) - Cochabamba
Pasante - Área Industrial - Santa Cruz
Ejecutivo(a) de Ventas – Canal Mayorista
Analista de Organización y Método
Pasante de Servicio Técnico - Cochabamba
Consignatario(a) / Administrador(a) de Tienda - Santa Cruz
Gerente Financiero(a)
Encargado(a) de Activos Fijos
Auxiliar Contable
Contador(a) Júnior - Santa Cruz
Vendedor(a) de Sala
Auxiliar de Comercio Exterior - Santa Cruz
Contador(a)
Representante Comercial
Contador(a) General para la Asociación Pro Arte y Cultura (APAC)
Auxiliar Contable
Auxiliar Contable
Supervisor(a) de Obras Civiles
Secretaria Recepcionista
Asistente Administrativo(a) Contable
Encargado(a) de Almacén
Analista Recursos Humanos - Oruro
Ejecutivos(as) de Ventas
Encargado(a) de Plataforma - Santa Cruz - Agencia Central
Analista de Cuentas Especiales - Beni
Analista de Cuentas Especiales - Santa Cruz
Analista de Cuentas Especiales - Cochabamba
Supervisor(a) de Almacén - Santa

Secretario(a) Administrativo(a) - Santa Cruz
Ejecutivas(os) de Ventas - Santa Cruz
Ejecutivo(a) de Ventas
Ejecutivo(a) de Cuentas, Eventos ATL y BTL - Santa Cruz
Encargado(a) de Fábrica - Ivirgarzama
Contador(a) General (Planta Achocalla - La Paz)
Desarrolladores(as) PHP, JavaScript, React Native - La Paz
Ejecutivo(a) de Ventas
Auxiliar Contable
Contador(a) General
Asesor(a) Jurídico(a) Comercial - La Paz
Ejecutivo(a) Comercial - Santa Cruz
Implementador(a) (Analista Funcional) - Santa Cruz
Ejecutivo(a) Comercial - Santa Cruz
Pasante Contable
Vendedor(a) de Planta
Asistente Administrativo(a) - Santa Cruz
Auxiliar de Activos Fijos - Santa Cruz
Analista de Organización y Métodos - Santa Cruz
Auditor(a) Júnior
Community Manager
Instaladores de Redes y Telecomunicaciones
Asesores(as) de Créditos - El Alto
Jefe de Agencia - Tarija
Ejecutivo(a) de Ventas - Santa Cruz
Mecánico Industrial
Encargado(a) Comercial - Tarija
Coordinador(a) de Gestión Técnica
Comunicador(a) Audiovisual Digital
Ingen

In [23]:
import csv
outfile = open("/Users/Roberto/Documents/Python Scripts/Machine Learning Projects/Labor Market/trabajos.csv", 'a',newline='')
writer = csv.writer(outfile)
try:
    for i in range(len(Results)):
        csvRow = []
        
        if Results[i][0] is not None:
            
            for cell in Results[i][0]:
                #print(cell)
                csvRow.append(cell)
                csvRow.append(Results[i][1])
                csvRow.append(Results[i][2])
                #print(csvRow)
            writer.writerow(csvRow)
finally:
    outfile.close() 

In [13]:
len(Results)

854

In [24]:
Results

[(['TIGO',
   '705134',
   'Santa Cruz de la Sierra, Bolivia',
   'Informática,Ingeniería ',
   'No Declarado ',
   '12 Enero 2019',
   '11 Febrero 2019',
   'datawarehouse engineer objetivo del cargoatender solicitudes requerimientos de integracion al dwh analisis de informacion elaboracion de reportes para los diferentes sistemas y analisis de la compania que te pedimos titulado en ingenieria de sistemas ingenieria informatica ingenieria en telecomunicaciones yo ramas afines experiencia de 3 anos en cargos similares conocimientos en inteligencia de negocios conocimiento en manejo de base de datos oracle sql server conocimiento en elaboracion de etl extraccion transformacional y lectura manejo de sistemas operativos linux windows tus competencias comunicacion eficaz solucion de problemas pensamiento analitico y conceptual capacidad de trabajo bajo presion planificacion y organizacion si te interesa esta posicion sube tu hoja de vida al siguiente linkhttpsmillicomuniversitycsodcomatsca